In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/18 00:47:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.rdd

MapPartitionsRDD[7] at javaToPython at NativeMethodAccessorImpl.java:0

In [4]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [5]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 4, 5, 54), PULocationID=129, total_amount=8.51),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 17, 19, 33, 5), PULocationID=75, total_amount=18.36),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 12, 41), PULocationID=117, total_amount=43.91),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 21, 25, 44), PULocationID=41, total_amount=12.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 21, 45, 19), PULocationID=129, total_amount=7.8)]

In [16]:
from datetime import datetime

In [17]:
start = datetime(year=2020, month=1, day=1)


In [8]:
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [9]:
rdd \
    .filter(filter_outliers) \
    .take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 4, 5, 54), PULocationID=129, total_amount=8.51),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 17, 19, 33, 5), PULocationID=75, total_amount=18.36),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 12, 41), PULocationID=117, total_amount=43.91),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 11, 21, 25, 44), PULocationID=41, total_amount=12.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 21, 45, 19), PULocationID=129, total_amount=7.8)]

In [10]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    PULocationID = row.PULocationID
    key = (hour, PULocationID)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [11]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(5)

[((datetime.datetime(2020, 1, 11, 4, 0), 129), (8.51, 1)),
 ((datetime.datetime(2020, 1, 17, 19, 0), 75), (18.36, 1)),
 ((datetime.datetime(2020, 1, 30, 12, 0), 117), (43.91, 1)),
 ((datetime.datetime(2020, 1, 11, 21, 0), 41), (12.3, 1)),
 ((datetime.datetime(2020, 1, 4, 21, 0), 129), (7.8, 1))]

In [12]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [18]:
from collections import namedtuple

In [19]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'PULocationID', 'revenue', 'count'])

In [20]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        PULocationID=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [21]:
from pyspark.sql import types

In [22]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [23]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [25]:
df_result.take(5)

24/12/18 00:51:20 ERROR Executor: Exception in task 0.0 in stage 8.0 (TID 12)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 812, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
           ^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'PySparkRuntime

Py4JJavaError: An error occurred while calling o173.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 12) (me-zoomcamp.us-central1-a.c.supple-comfort-444210-e6.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 812, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
           ^^^^^^^^^^^^^^^
  File "/usr/lo

In [ ]:
df_result.write.parquet('tmp/green-revenue')